# Setting up

Import all the neccessary modules

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

print("Setup Complete")

Setup Complete


Get the dataframe for analysis

In [ ]:
journal = "/content/journal.csv"

In [ ]:
journal_detail = "/content/journal_detail.csv"

In [ ]:
journal_df = pd.read_csv(journal, sep= ";")

In [ ]:
journal_detail_df = pd.read_csv(journal_detail, sep=";")

In [ ]:
df = pd.merge(journal_df,journal_detail_df, on="oid")

In [ ]:
#Check

# journal_detail_df

In [ ]:
#check merged file

# df

# Data Pre-processing and Cleaning


set to datetime format


In [ ]:
df['journaldatetime'] = pd.to_datetime(df['journaldatetime'])

Calculate z-score of amount column

In [ ]:
z_score = np.abs((df['amount'] - df['amount'].mean()) / df['amount'].std())

Clean the data

In [ ]:
# Identify outlier rows where z-score is greater than 3
outlier_rows = df[z_score > 3].index

# Remove outlier rows and rows where productoid is 8
df = df.drop(outlier_rows, axis=0)
df = df[df['productoid'] != 8]

# Recalculate z-score and identify remaining outlier rows
z_score = np.abs((df['amount'] - df['amount'].mean()) / df['amount'].std())
outlier_rows = df[z_score > 3].index

# Remove any remaining outlier rows
df = df.drop(outlier_rows, axis=0)

See the shape of the dataframe after cleaning

In [ ]:
# Print the shape of the DataFrame
print(df.shape)

(675039, 37)


# Data Exploration

In [ ]:
# Convert the datetimejournal variable to datetime format
df['journaldatetime'] = pd.to_datetime(df['journaldatetime'])

In [ ]:
df

,oid,journaldatetime,posnumber,userid,document_no,customeroid,qty_x,amount,discount_x,total_x,total_receive,change,period_num,shift_id,print_count,journal_type,reference_no,total_goods_amount,total_tax_amount,journaloid,productoid,unitoid,unitprice,qty_y,amout,discount_y,total_y,item_type,status,pumpnumber,hosenumber,delivery_id,item_name,goods_amout,tax_type,tax_rate,tax_amout
0,658440,2019-03-03 15:41:50.000,1,Cashier2,2019030000658430,1,2.70,75.0,0.0,75.0,75.0,0.0,764,2,1,1,NaN,70.093,4.907,593773,1006,1,25.55,2.94,75.1,0.0,75.1,2,1,2.0,2.0,725164.0,GASOHOL91,70.187,2.0,7.0,4.913
1,658441,2019-03-03 15:42:51.218,1,Cashier2,2019030000658431,1,32.25,900.0,0.0,900.0,900.0,0.0,764,2,1,1,NaN,841.121,58.879,593773,1003,1,25.82,2.71,70.0,0.0,70.0,2,1,2.0,3.0,725165.0,GASOHOL95,65.421,2.0,7.0,4.579
2,658442,2019-03-03 15:44:55.562,1,Cashier2,2019030000658432,1,10.81,300.0,0.0,300.0,300.0,0.0,764,2,1,1,NaN,280.374,19.626,593774,1005,1,28.73,48.73,1400.0,0.0,1400.0,2,1,7.0,2.0,725167.0,DIESEL PLUS,1308.411,2.0,7.0,91.589
4,658444,2019-03-03 15:46:17.406,1,Cashier2,2019030000658434,1,10.81,300.0,0.0,300.0,300.0,0.0,764,2,1,1,NaN,280.374,19.626,593776,1000,1,24.86,20.11,500.0,0.0,500.0,2,1,7.0,1.0,725168.0,DIESEL,467.290,2.0,7.0,32.710
5,658445,2019-03-03 15:46:29.265,1,Cashier2,2019030000658435,1,10.80,300.0,0.0,300.0,300.0,0.0,764,2,1,1,NaN,280.374,19.626,593777,1003,1,25.82,3.09,80.0,0.0,80.0,2,1,2.0,3.0,725169.0,GASOHOL95,74.766,2.0,7.0,5.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713851,658434,2019-03-03 15:32:53.750,1,Cashier2,2019030000658424,1,33.15,920.0,0.0,920.0,920.0,0.0,764,2,1,1,NaN,859.813,60.187,593767,1006,1,25.55,3.92,100.2,0.0,100.2,2,1,2.0,2.0,725103.0,GASOHOL91,93.645,2.0,7.0,6.555
713853,658436,2019-03-03 15:34:08.828,1,Cashier2,2019030000658426,1,26.30,730.0,0.0,730.0,730.0,0.0,764,2,1,1,NaN,682.243,47.757,593769,1000,1,24.86,28.15,700.0,0.0,700.0,2,1,7.0,1.0,725160.0,DIESEL,654.206,2.0,7.0,45.794
713854,658437,2019-03-03 15:34:59.921,1,Cashier2,2019030000658427,1,10.80,300.0,0.0,300.0,300.0,0.0,764,2,1,1,NaN,280.374,19.626,593770,1006,1,25.55,2.34,60.0,0.0,60.0,2,1,5.0,2.0,725161.0,GASOHOL91,56.075,2.0,7.0,3.925
713855,658438,2019-03-03 15:39:27.921,1,Cashier2,2019030000658428,1,10.81,300.0,0.0,300.0,300.0,0.0,764,2,1,1,NaN,280.374,19.626,593771,1000,1,24.86,28.15,700.0,0.0,700.0,2,1,7.0,1.0,725163.0,DIESEL,654.206,2.0,7.0,45.794


# ---


In [ ]:
df=df.loc[:,['journaldatetime','amout','unitprice','qty_y','productoid']]


In [ ]:
df=df.rename(columns={'journaldatetime':'date-time','qty_y': 'qty','amout':'amount', 'productoid':'productid'})
df.head()

,date-time,amount,unitprice,qty,productid
0,2019-03-03 15:41:50.000,75.1,25.55,2.94,1006
1,2019-03-03 15:42:51.218,70.0,25.82,2.71,1003
2,2019-03-03 15:44:55.562,1400.0,28.73,48.73,1005
4,2019-03-03 15:46:17.406,500.0,24.86,20.11,1000
5,2019-03-03 15:46:29.265,80.0,25.82,3.09,1003


##Random Forest **Regressor**

First, we format date-time to get month and day values

In [ ]:
df['date-time'] = pd.to_datetime(df['date-time'])
df['month'] = df['date-time'].dt.month
df['day'] = df['date-time'].dt.day

In [ ]:
# df

First, the input features (month, day, product ID, and unit price) are extracted from the DataFrame df, and the target variable (quantity) is assigned to y.

```
# This is formatted as code
```



In [ ]:
X = df[['month', 'day','productid','unitprice']]
y = df['qty']

Then, the data is split into training and testing sets using the train_test_split function from the scikit-learn library, with a test size of 0.2 and a random state of 69

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

Next, a random forest regressor model is initialized with 100 estimators and a random state of 69, and then trained on the training data using the fit method.

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=69)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=69)

The predict method is then used to generate predicted values of the quantity for the test set, which are assigned to y_pred.

In [ ]:
y_pred = rf.predict(X_test)

Finally, the mean absolute error (MAE) and R-squared value are calculated using the mean_absolute_error and r2_score functions from the scikit-learn library, respectively.

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

MAE: 11.39334316981484


In [ ]:
r2 = r2_score(y_test, y_pred)
print("R-squared value:", r2)

R-squared value: 0.20896466624716292


#Another Model

Now, I will be attempting to use a Gradient Boosting Regressor to predict the quantity of products sold based on my selected features.

First, we import the neccessary modules

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

Then, the input features are extracted from df and stored in X. The target variable "qty" is also extracted from df and stored in the variable "y".
After that, the dataset is split into training and testing sets using "train_test_split" with a test size of 0.2.

In [ ]:
X = df[['month', 'day','productid','unitprice']]
y = df['qty']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


The Gradient Boosting Regressor model is initialized accordingly with hyperparameters. The model is then trained on the training data using the "fit" method.

In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=42)
gb.fit(X_train, y_train)


GradientBoostingRegressor(max_depth=1, random_state=42)

Predictions are made on the test data using the "predict" method and MSE, MAE, and R-squared value of the model's predictions are calculated using its respective functions.

In [ ]:
predictions = gb.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print('MSE:', mse)

mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

r2 = r2_score(y_test, y_pred)
print("R-squared value:", r2)

MSE: 380.51382373399156
MAE: 15.610418520314331
R-squared value: -0.22973241198583305
